In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
import torch.nn as nn
from torch_geometric.data import DataLoader
import argparse
import numpy as np
import random
import ogb
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
import torch.nn.functional as F
from graph_transformer import GT
import os
import datetime
from tqdm import tqdm

parser = argparse.ArgumentParser(description='PyTorch implementation of relative positional encodings and relation-aware self-attention for graph Transformers')
args = parser.parse_args("")
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("device:", args.device)

RDKit WARNING: [20:11:48] Enabling RDKit 2019.09.3 jupyter extensions
In /opt/conda/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/conda/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/conda/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /opt/conda/lib/python3.6/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /opt/conda/lib/python3.6/site-packa

device: cuda


In [2]:
args.dataset = 'ogbg-molhiv'
args.n_classes = 1
args.lr = 2e-4
args.n_hid = 512
args.n_heads = 8
args.n_layer = 4
args.dropout = 0.2
args.num_epochs = 100
args.k_hop_neighbors = 3
args.weight_decay = 1e-2
args.node_dim = 9
args.edge_dim = 3
args.bsz      = 128

In [3]:
import torch
from networkx.algorithms.shortest_paths.generic import shortest_path
from networkx.algorithms.approximation.connectivity import all_pairs_node_connectivity
from networkx.algorithms.clique import node_clique_number
from networkx.algorithms.centrality import betweenness_centrality, edge_betweenness_centrality
from torch_geometric.utils.convert import to_networkx
from torch_geometric.data import Data
from torch_geometric.utils import to_dense_adj, dense_to_sparse

def pre_process(d):
    node_size = d.x.size(0)
    #     TODO: add summary node that connects to all the other nodes.
    
    #     Construct networkX type of original graph for different metrics
    d_nx = to_networkx(d, to_undirected=True)
    
    #     Augment the graph to be K-hop graph
    dense_orig_adj = to_dense_adj(d.edge_index, max_num_nodes=node_size).squeeze(dim=0)
    for k in range(args.k_hop_neighbors):
        dense_orig_adj = F.hardtanh(torch.mm(dense_orig_adj, dense_orig_adj))
    d.edge_index = dense_to_sparse(dense_orig_adj)[0]
    
    #     Calculate structural feature by the ORIGNAL graph, add them to new edge set.
    sd_edge_attr = shortest_distances(d_nx, d.edge_index)
    cn_edge_attr = node_connectivity(d_nx, d.edge_index)
    return Data(x=d.x, y=d.y, edge_index=d.edge_index, edge_attr=d.edge_attr, \
         sd_edge_attr=sd_edge_attr, cn_edge_attr=cn_edge_attr)
    
def shortest_distances(d_nx, edge_index):
    edge_attr = []
    p = shortest_path(d_nx)
    for s, t in edge_index.t().tolist():
        if s in p and t in p[s]:
            edge_attr += [len(p[s][t]) - 1]
        else:
            edge_attr += [0]
        
    return torch.LongTensor(edge_attr)

def node_connectivity(d_nx, edge_index):
    edge_attr = []
    p = all_pairs_node_connectivity(d_nx)
    for s, t in edge_index.t().tolist():
        if s in p and t in p[s]:
            edge_attr += [p[s][t]]
        else:
            edge_attr += [0]
    return torch.LongTensor(edge_attr)

def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.inits import glorot, uniform
from torch_geometric.utils import softmax
from ogb.graphproppred.mol_encoder import AtomEncoder, BondEncoder
import math
from torch_geometric.nn import global_mean_pool


class RelEncoding(nn.Module):
    def __init__(self, n_hid, max_len = 240, dropout = 0.2):
        super(RelEncoding, self).__init__()
        self.emb = nn.Embedding(max_len, n_hid)
        self.drop = nn.Dropout(dropout)
        self.emb.weight.data.uniform_(-0.1, 0.1)
    def forward(self, t):
        return self.drop(self.emb(t))

class GT(nn.Module):
    def __init__(self, n_hid, n_out, n_heads, n_layers, edge_dim_dict, dropout = 0.2):
        super(GT, self).__init__()
        self.node_encoder = AtomEncoder(emb_dim=n_hid)
        self.n_hid     = n_hid
        self.n_out     = n_out
        self.drop      = nn.Dropout(dropout)
        self.gcs       = nn.ModuleList([GT_Layer(n_hid, n_heads, edge_dim_dict, dropout)\
                                      for _ in range(n_layers)])
        self.out       = nn.Linear(n_hid, n_out)

    def forward(self, node_attr, edge_index, cn_edge_attr, sd_edge_attr, batch_idx):
        node_rep = self.node_encoder(node_attr)
        for gc in self.gcs:
            node_rep = gc(node_rep, edge_index, cn_edge_attr, sd_edge_attr)
        return self.out(global_mean_pool(node_rep, batch_idx))  

class GT_Layer(MessagePassing):
    def __init__(self, n_hid, n_heads, edge_dim_dict, dropout = 0.2, **kwargs):
        super(GT_Layer, self).__init__(node_dim=0, aggr='add', **kwargs)

        self.n_hid         = n_hid
        self.n_heads       = n_heads
        self.d_k           = n_hid // n_heads
        self.sqrt_dk       = math.sqrt(self.d_k)
        self.att           = None
        
        
        self.k_linear   = nn.Linear(n_hid,   n_hid)
        self.q_linear   = nn.Linear(n_hid,   n_hid)
        self.v_linear   = nn.Linear(n_hid,   n_hid)
        self.a_linear   = nn.Linear(n_hid,   n_hid)
        self.norm       = nn.LayerNorm(n_hid)
        self.drop       = nn.Dropout(dropout)
        
        self.struc_enc = nn.ModuleDict({
            key : RelEncoding(max_len = edge_dim_dict[key], n_hid = n_hid, dropout = dropout)
                for key in edge_dim_dict
        })
        
        self.mid_linear  = nn.Linear(n_hid,  n_hid * 2)
        self.out_linear  = nn.Linear(n_hid * 2,  n_hid)
        self.out_norm    = nn.LayerNorm(n_hid)
        
    def forward(self, node_inp, edge_index, cn_edge_attr, sd_edge_attr):
        return self.propagate(edge_index, node_inp=node_inp, \
                              cn_edge_attr = cn_edge_attr, sd_edge_attr = sd_edge_attr)

    def message(self, edge_index_i, node_inp_i, node_inp_j, cn_edge_attr, sd_edge_attr):
        '''
            j: source, i: target; <j, i>
        '''
        data_size = edge_index_i.size(0)
        '''
            Create Attention and Message tensor beforehand.
        '''
                
        target_node_vec = node_inp_i
        source_node_vec = node_inp_j + self.struc_enc['cn'](cn_edge_attr) + self.struc_enc['sd'](sd_edge_attr)

        q_mat = self.q_linear(target_node_vec).view(-1, self.n_heads, self.d_k)
        k_mat = self.k_linear(source_node_vec).view(-1, self.n_heads, self.d_k)
        v_mat = self.v_linear(source_node_vec).view(-1, self.n_heads, self.d_k)
        
        '''
            Softmax based on target node's id (edge_index_i). Store attention value in self.att for later visualization.
        '''
        self.att = self.dropout(softmax((q_mat * k_mat).sum(dim=-1) / self.sqrt_dk, edge_index_i))
        res = v_mat * self.att.view(-1, self.n_heads, 1)
        return res.view(-1, self.n_hid)


    def update(self, aggr_out, node_inp):
        trans_out = self.norm(self.drop(self.a_linear(F.gelu(aggr_out))) + node_inp)
        trans_out = self.out_norm(self.drop(self.out_linear(F.gelu(self.mid_linear(trans_out)))) + trans_out)
        return trans_out

In [5]:
print("Loading data...")
print("dataset: {} ".format(args.dataset))
dataset = PygGraphPropPredDataset(name=args.dataset, pre_transform=pre_process)
evaluator = Evaluator(name=args.dataset)
split_idx = dataset.get_idx_split()
edge_dim_dict = {'sd': dataset.data.sd_edge_attr.max().int().item() + 1, \
                 'cn': dataset.data.cn_edge_attr.max().int().item() + 1}
model = GT(args.n_hid, args.n_classes, args.n_heads, args.n_layer, edge_dim_dict, args.dropout).to(args.device)

Loading data...
dataset: ogbg-molhiv 


In [6]:
from transformers.optimization import AdamW
def get_optimizer(model: nn.Module, learning_rate: float = 1e-4, adam_eps: float = 1e-6,
                  weight_decay: float = 0.0, ) -> torch.optim.Optimizer:
    no_decay = ['bias', 'LayerNorm.weight']

    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate, eps=adam_eps)
    return optimizer

In [7]:
print('Model #Params: %d' % get_n_params(model))

criterion = torch.nn.BCEWithLogitsLoss(reduction = "mean")

optimizer = get_optimizer(model, weight_decay = args.weight_decay)

train_loader = DataLoader(dataset[split_idx["train"]], batch_size=args.bsz, shuffle=True)
valid_loader = DataLoader(dataset[split_idx["valid"]], batch_size=args.bsz, shuffle=False)
test_loader  = DataLoader(dataset[split_idx["test"]],  batch_size=args.bsz, shuffle=False)

scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=args.lr, pct_start = 0.05,\
        steps_per_epoch=len(train_loader), epochs = args.num_epochs, anneal_strategy = 'linear')

Model #Params: 8530945


In [ ]:
stats = []
for epoch in range(args.num_epochs):
    model.train()
    train_loss = []
    for data in tqdm(train_loader):
        data.to(args.device)
        out = model(data.x, data.edge_index, data.cn_edge_attr, data.sd_edge_attr, data.batch)
        loss = criterion(out, data.y.float())
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        optimizer.zero_grad()
        scheduler.step()
        
        train_loss += [loss.item()]

    model.eval()
    with torch.no_grad():
        valid_loss = []
        y_true = []
        y_scores = []
        for data in valid_loader:
            data.to(args.device)
            out = model(data.x, data.edge_index, data.cn_edge_attr, data.sd_edge_attr, data.batch)

            loss = criterion(out, data.y.float())
            valid_loss += [loss.item()]

            y_true += [data.y]
            y_scores += [out]

        input_dict = {"y_true": torch.cat(y_true), "y_pred": torch.cat(y_scores)}
        valid_rocauc = evaluator.eval(input_dict)['rocauc']
        
        
        test_loss = []
        y_true = []
        y_scores = []
        for data in test_loader:
            data.to(args.device)
            out = model(data.x, data.edge_index, data.cn_edge_attr, data.sd_edge_attr, data.batch)

            loss = criterion(out, data.y.float())
            test_loss += [loss.item()]

            y_true += [data.y]
            y_scores += [out]

        input_dict = {"y_true": torch.cat(y_true), "y_pred": torch.cat(y_scores)}
        test_rocauc = evaluator.eval(input_dict)['rocauc']
    
    print('Epoch %d: LR: %.5f, Train loss: %.3f Valid loss: %.3f  Valid ROC-AUC: %.3f Test loss: %.3f  Test ROC-AUC: %.3f' \
          % (epoch, optimizer.param_groups[0]['lr'], np.average(train_loss), np.average(valid_loss), \
            valid_rocauc, np.average(test_loss), test_rocauc))
    stats += [[epoch, np.average(train_loss), np.average(valid_loss), valid_rocauc, np.average(test_loss), test_rocauc]]

  0%|          | 0/258 [00:00<?, ?it/s]

Epoch 0: LR: 0.00005, Train loss: 0.171 Valid loss: 0.101  Valid ROC-AUC: 0.659 Test loss: 0.136  Test ROC-AUC: 0.642


  0%|          | 0/258 [00:00<?, ?it/s]

Epoch 1: LR: 0.00008, Train loss: 0.152 Valid loss: 0.087  Valid ROC-AUC: 0.718 Test loss: 0.127  Test ROC-AUC: 0.671


  0%|          | 0/258 [00:00<?, ?it/s]

...

  0%|          | 0/258 [00:00<?, ?it/s]

Epoch 71: LR: 0.00006, Train loss: 0.006 Valid loss: 0.223  Valid ROC-AUC: 0.798 Test loss: 0.398  Test ROC-AUC: 0.779


  0%|          | 0/258 [00:00<?, ?it/s]

Epoch 72: LR: 0.00006, Train loss: 0.006 Valid loss: 0.220  Valid ROC-AUC: 0.787 Test loss: 0.366  Test ROC-AUC: 0.782


 52%|█████▏    | 135/258 [00:27<00:27,  4.51it/s]